In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
%load_ext qat.core.magic
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

In [ ]:
import copy

In [ ]:
from QuantumMultiplexors_Module_examples import expectation_loading_data
from PhaseAmplification_Module import load_q_gate

In [ ]:
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 10
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

## Circuit

In [ ]:
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
Q_Gate = load_q_gate(P_Gate, R_gate)

In [ ]:
def Do(q_prog, q_gate, n_ops, nbshots=1000):
    q_bits = q_prog.registers[0]
    for i in range(n_ops):
        q_prog.apply(q_gate, q_bits)
    circuit = q_prog.to_circ(submatrices_only=True)
    job = circuit.to_job(qubits=[len(q_bits)-1], nbshots = nbshots)
    return job, circuit

In [ ]:
K=10 
pdf = []
ListOfCircutis = []
for m_k in range(K):
    step_job, step_cricuit = Do(copy.deepcopy(Qprog), Q_Gate, m_k)
    ListOfCircutis.append(step_cricuit)
    step_result = run_job(lineal_qpu.submit(step_job))
    step_pdf = postprocess_results(step_result)
    step_pdf['K'] = m_k
    pdf.append(step_pdf)
pdf = pd.concat(pdf)

In [ ]:
cir = ListOfCircutis[1]
%qatdisplay cir

In [ ]:
def get_sub_pdf(input_pdf, state):
    pdf = input_pdf.copy(deep=True)
    phi = pdf[pdf['States'] == state]
    phi.reset_index(drop=True, inplace=True)
    phi.drop(columns=['Amplitude', 'States'], inplace = True)
    phi.rename(
        columns = {'Probability': 'Probability_{}'.format(state)},
        inplace = True
    )
    return phi.drop_duplicates()
    return phi

In [ ]:
pdf_bis = (get_sub_pdf(pdf, '|0>')).merge(get_sub_pdf(pdf, '|1>'), on='K')

In [ ]:
pdf_bis

In [ ]:
def p_phi1(x, theta):
    return np.sin((2*x+1)*theta)**2
def p_phi0(x, theta):
    return np.cos((2*x+1)*theta)**2

In [ ]:
import scipy.optimize as so

In [ ]:
def Loss(theta, x, y):
    y_hat = np.sin((2*x+1)*theta)**2
    return np.mean((y_hat-y)**2)
    
    

In [ ]:
r = so.minimize(
    Loss,
    [np.pi/4],
    (pdf_bis['K'], pdf_bis['Probability_|1>']),
    #method='Newton-CG',
    bounds=[(0., np.pi/2)],
    tol = 1e-6,
    options={'disp': True}
)
print('theta: {}'.format(r.x[0]))

In [ ]:
so.minimize?

In [ ]:
%matplotlib inline
plt.plot(pdf_bis['K'], pdf_bis['Probability_|1>'], 'o')
#x = np.linspace(0, 10, num=100)
plt.plot(pdf_bis['K'], p_phi1(pdf_bis['K'], r.x[0]))

In [ ]:
sum(p_X*f_X)

In [ ]:
so.curve_fit()

In [ ]:
K=10 
pdf = []
for k in range(1, K+1):
    step_job, step_cricuit = Do(copy.deepcopy(Qprog), Q_Gate, k)
    step_result = run_job(lineal_qpu.submit(step_job))
    step_pdf = postprocess_results(step_result)
    step_pdf['K'] = k
    pdf.append(step_pdf)
pdf = pd.concat(pdf)

In [ ]:
step_pdf['K'] = k

In [ ]:
step_pdf

In [ ]:
j, c = Do(copy.deepcopy(Qprog), Q_Gate, 1)
res = run_job(lineal_qpu.submit(j))
a = postprocess_results(res)

In [ ]:
a.iloc[1]['Probability']

In [ ]:
%qatdisplay c

In [ ]:
K = 10

Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
qbits = Qprog.registers[0]
for k in range(K):
    Qprog.apply(Q_Gate, qbits)
circuit = Qprog.to_circ(submatrices_only=True)
%qatdisplay circuit

In [ ]:
Q_Gate.